<a href="https://colab.research.google.com/github/Ravgeer/sparkscon2024/blob/main/Sparkscon2024_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **1. Clone Data Directory, Pretrained Models, Scripts and Augmentations**


In [ ]:
!git clone https://github.com/Ravgeer/sparkscon2024

Cloning into 'sparkscon2024'...
remote: Enumerating objects: 5731, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 5731 (delta 1), reused 0 (delta 0), pack-reused 5722
Receiving objects: 100% (5731/5731), 210.20 MiB | 39.46 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (5751/5751), done.


---
# **2. Clone and Install YOLO and download pre-trained models for Classification**


In [3]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.7/201.2 GB disk)


In [4]:
from utils.downloads import attempt_download
p5 = ['n','s','l','m','x']

cls = [f'{x}-cls' for x in p5]

for x in cls:
  attempt_download(f'weights/yolov5{x}.pt')

100%|██████████| 4.87M/4.87M [00:00<00:00, 75.9MB/s]

100%|██████████| 10.5M/10.5M [00:00<00:00, 167MB/s]

100%|██████████| 50.9M/50.9M [00:00<00:00, 195MB/s]

100%|██████████| 24.9M/24.9M [00:00<00:00, 111MB/s]

100%|██████████| 92.0M/92.0M [00:00<00:00, 327MB/s]



---
# **3. Generate Augmentations**


In [ ]:
### Python Script for image scraping from Weblinks
!python /content/yolov5/sparkscon2024/scripts/scrape_images.py 'https://www.shutterstock.com/search/granny-smith?image_type=photo' '/content/scrape_images/apples/granny_smith_new' --delay 1.0


In [ ]:
### Python Script for image Augmentation from data Source
!python /content/yolov5/sparkscon2024/scripts/augmented_images.py '/content/sparkscon2024/scrape_images/apples/train/braeburn' '/content/sparkscon2024/scrape_images/apples/train/braeburn_new' --num_augmentations 2

---
# **4. Image Classification Training**




In [ ]:
#Connect G-Drive to exchange data
from google.colab import drive
drive.mount('/content/drive')

In [5]:
### set optimal_batch_size
import torch
from utils.torch_utils import select_device
from utils.autobatch import check_train_batch_size

def find_optimal_batch(weights, imgsz=640, amp=True):
    # Select the device (GPU or CPU)
    device = select_device('')
    print(f"selected_device: {device}")
    # Load the model
    model = torch.load(weights, map_location=device)['model'].float()  # Load FP32 model
    model.fuse()  # Fuse Conv2d + BatchNorm2d layers
    if amp:
        model.half()  # Convert model to FP16 if amp (automatic mixed precision) is enabled

    # Find optimal batch size
    return check_train_batch_size(model, imgsz=imgsz, amp=amp)

if __name__ == '__main__':
    weights = '/content/yolov5/weights/yolov5l-cls.pt'  # Path to your weights file
    imgsz = 640  # Image size
    amp = True  # Automatic Mixed Precision (AMP)

    optimal_batch_size = find_optimal_batch(weights, imgsz, amp)
    print(f"Optimal batch size: {optimal_batch_size}")


YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)



selected_device: cuda:0


Fusing layers... 
Model summary: 215 layers, 26535976 parameters, 0 gradients, 69.3 GFLOPs
AutoBatch: Computing optimal batch size for --imgsz 640
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.38G reserved, 0.23G allocated, 14.14G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    26535976       68.32         0.407         13.37           nan        (1, 3, 640, 640)               (1, 1000)
    26535976       136.6         0.354         23.89           nan        (2, 3, 640, 640)               (2, 1000)
    26535976       273.3         0.449         35.88           nan        (4, 3, 640, 640)               (4, 1000)
    26535976       546.5         0.646         54.59           nan        (8, 3, 640, 640)               (8, 1000)


AutoBatch: Using batch-size 242 for CUDA:0 11.90G/14.75G (81%) ✅


    26535976        1093         1.040         99.62           nan       (16, 3, 640, 640)              (16, 1000)
Optimal batch size: 242


In [6]:
## optimal_batch_size ##
!python -W ignore classify/train.py --batch {optimal_batch_size} --epochs 2 --data  /content/sparkscon2024/scrape_images/apples/ --model yolov5s-cls.pt --name yolov5s_results


2024-06-19 14:28:45.785250: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 14:28:45.785318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 14:28:45.889222: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5s-cls.pt, data=/content/sparkscon2024/scrape_images/apples/, epochs=2, batch_size=242, imgsz=224, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=yolov5s_results, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, loc

In [ ]:
### User defined batch size ##
!python -W ignore classify/train.py --img 640 --batch 16 --epochs 2 --data  /content/sparkscon2024/scrape_images/apples/ --model yolov5l-cls.pt --name yolov5l_results


In [ ]:
#### We can copy our trained models in Google drive with this command
!cp -r /content/yolov5/runs/train-cls/  /content/drive/MyDrive/

---
# **5. Test Custom Model using Test Images**


In [10]:
!python classify/predict.py --weights /content/sparkscon2024/models/weights/best.pt --source /content/sparkscon2024/scrape_images/apples/test/jonagored/pear-amidst-red-jonagored-apples-260nw-2389319923.jpg --save-txt

### yolo_models_path:     /content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt
### standard yolo model:  yolov5l-cls.pt

classify/predict: weights=['yolov5l-cls.pt'], source=/content/sparkscon2024/scrape_images/apples/test/jonagored/pear-amidst-red-jonagored-apples-260nw-2389319923.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=True, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100% 50.9M/50.9M [00:00<00:00, 295MB/s]

Fusing layers... 
Model summary: 215 layers, 26535976 parameters, 0 gradients, 69.3 GFLOPs
image 1/1 /content/sparkscon2024/scrape_images/apples/test/jonagored/pear-amidst-red-jonagored-apples-260nw-2389319923.jpg: 224x224 Granny Smith 0.79, banana 0.05, grocery store 0.04, lemon 0.02, strawberry 0.01, 8.8ms
Speed: 0.3ms pre-process, 8.8ms inference, 38.1ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp4
1 labels saved to runs/predict-cl

---
# **6. Export model**


In [11]:
!python export.py --weights /content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt --include onnx

export: data=data/coco128.yaml, weights=['/content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.12 torch-2.3.0+cu121 CPU

Traceback (most recent call last):
  File "/content/yolov5/export.py", line 940, in <module>
    main(opt)
  File "/content/yolov5/export.py", line 935, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/export.py", line 798, in run
    model = attempt_load(weights, device=device, inplace=True, fuse=True)  # load FP32 model
  File "/content/yolov5/models/experimental.

---
# **7. Load Model**


In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

Ultralytics YOLOv8.2.35 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)


Fusing layers... 
Model summary: 215 layers, 25258819 parameters, 0 gradients, 68.3 GFLOPs


AttributeError: 'ClassificationModel' object has no attribute 'yaml'

---
# **Archive**


In [ ]:
#Connect G-Drive to exchange data
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## required for Autobatch
%cd ..
!git clone https://github.com/ultralytics/ultralytics.git

In [ ]:
###!python /content/yolov5/ultralytics/ultralytics/utils/autobatch.py

In [ ]:
%pwd

In [ ]:
!rm -r '/content/scrape_images'